# Changing Parameters of the Taxes and Transfers System

This tutorial focuses on the `policy_params` of GETTSIM, one of the two parts that compose a `PolicyEnvironment`.. It is a dictionary that contains all the date-specific parameters (e.g. level of unemployment benefit) that are necessary to compute the target variables we are interested in. GETTSIM not only provides current and past policy environments, but also allows users to alter policies for instance by changing parameters.

In [ ]:
import copy

import numpy as np
import pandas as pd
import plotly.express as px

from gettsim import (
    compute_taxes_and_transfers,
    create_synthetic_data,
    set_up_policy_environment,
)

## Example: Increasing Child Benefit

In this tutorial, we will implement such a policy change: a raise of the child benefit by 20 € for each child.

*Small Recap: Kindergeld is a child benefit that can be claimed by parents in Germany. Mainly lower-income families benefit from Kindergeld. For higher-income families, the tax credits for children are more advantageous, so that they do not receive Kindergeld. The child benefit and how they are implemented in GETTSIM is explained in more detail in the* [advanced usage tutorial](advanced_usage.ipynb).

To implement the raise in GETTSIM, we will change relevant parameters in the existing system by editing the policy parameters that shape the policy environment.

## Finding the Relevant Parameter

Firstly, we can load the existing policy environment for the year 2020 using the function `set_up_policy_environment`.

In [ ]:
environment = set_up_policy_environment(2020)

We can open the dictionary with all the parameters by typing `environment.params`. The parameters are saved to a nested dictionary, meaning that the dictionary consists of multiple further dictionaries. This can make displaying the parameter values look confusing at first sight.

To get an idea of the structure of the dictionary, we will work us through it step by step. First we take a look at the `keys` of the `dict`. The first layer of keys represent different policy groups.

In [ ]:
print(*environment.params.keys(), sep="\n")

Since we are interested in altering child benefits, we will select the key "kindergeld" to inspect the according parameters further. The parameters are saved to a sub-dictionary which can be selected using `environment.params["kindergeld"]`. It contains the keys to different parameter groups that affect child benefits in the German system:

In [ ]:
print(*environment.params["kindergeld"].keys(), sep="\n")

- **kindergeld_altersgrenze** is the maximum age of a child that is entitled to *Kindergeld*.

- **kindergeld** is the amount of money that parents receive for their children.

- **kindergeld_einkommensgrenze** is the maximum annual income of a child that is still entitled to *Kindergeld*.

- **kindergeld_stundengrenze** is the maximum number of weekly working hours of a child that is entitled to *Kindergeld*.

- **datum** specifies the date to which the parameters apply in the German taxes and transfers system. 

The reform we are simulating influences the amount of money that parents receive for their children. Let us take a look at the value of this parameter:

In [ ]:
environment.params["kindergeld"]["kindergeld"]

For the first and second child, the monthly *Kindergeld* is 204€. For the third child, monthly *Kindergeld* is 210€. For each additional child, *Kindergeld* is 235€.

## Changing the Parameter

To implement the bonus, we create a new parameter dictionary by creating a copy of the original which we can then alter.

In [ ]:
policy_params_new = copy.deepcopy(environment.params)

Using a loop, we add the 100€ to each entry of our new `policy_params_with_bonus["kindergeld"]["kindergeld"]`.

In [ ]:
# Loop through policy paramaters to add the special child bonus.
for n in policy_params_new["kindergeld"]["kindergeld"]:
    policy_params_new["kindergeld"]["kindergeld"][n] += 20

Now we can check if the ```["kindergeld"]["kindergeld"]``` entries in the new parameter dictionary (```policy_params_new```) are as expected:

In [ ]:
policy_params_new["kindergeld"]["kindergeld"]

It worked, we raised all values by 20!

## Applying the Edited Parameter Dictionary to Simulated Households

We use simulated data to illustrate the impact of the bonus we added to the *Kindergeld* parameters. For simplicity, we will only look at households with two children and two parents. 

In [ ]:
data = create_synthetic_data(
    n_adults=2,
    n_children=2,
    specs_heterogeneous={
        "einkommensteuer__einkünfte__aus_nichtselbstständiger_arbeit__bruttolohn_m": [
            [i, 0, 0, 0] for i in np.linspace(1000, 8000, 701)
        ]
    },
)

# Compute number of children in household and add it to data.
children = compute_taxes_and_transfers(
    data=data,
    environment=environment,
    targets="anz_kinder_bis_17_hh",
)
# Compute sum of pension contributions in household and add it to data.
sozialversicherung__rente__summe_private_gesetzliche_rente_m = (
    compute_taxes_and_transfers(
        data=data,
        environment=environment,
        targets="sozialversicherung__rente__summe_private_gesetzliche_rente_m",
    )
)
data["anz_kinder_bis_17_hh"] = children["anz_kinder_bis_17_hh"]
data["sozialversicherung__rente__summe_private_gesetzliche_rente_m"] = (
    sozialversicherung__rente__summe_private_gesetzliche_rente_m[
        "sozialversicherung__rente__summe_private_gesetzliche_rente_m"
    ]
)
data.head()

This simulated data set shows 701 households of two adults and their two children. The households only vary in their income.

First, we compute the *Kindergeld* with the original parameters:

In [ ]:
kindergeld_status_quo = compute_taxes_and_transfers(
    data=data,
    environment=environment,
    targets="kindergeld__betrag_m_hh",
)

kindergeld_status_quo[["kindergeld__betrag_m_hh"]]

Now we do exactly the same, but using the `policy_params_new`.

In [ ]:
environment_new = environment.replace_all_parameters(policy_params_new)
kindergeld_new = compute_taxes_and_transfers(
    data=data,
    environment=environment_new,
    targets="kindergeld__betrag_m_hh",
)
kindergeld_new[["kindergeld__betrag_m_hh"]]

## Compare Results

Using the results shown above, we can now easily visualize and compare the amount of *Kindergeld* for different income groups in the two scenarios. Therefore, we reorganize our data. We create a DataFrame with the total monthly income of each household and the *Kindergeld* they receive with the `policy_params_new` and the `environment.params`:

In [ ]:
# Group data by household id and sum the gross monthly income.
total_income_m_hh = data.groupby("hh_id")[
    "einkommensteuer__einkünfte__aus_nichtselbstständiger_arbeit__bruttolohn_m"
].sum()
total_income_m_hh.tail(10)

In [ ]:
# Create DataFrame with relevant columns for plotting.
df = pd.DataFrame()
df["Status Quo"] = kindergeld_status_quo["kindergeld__betrag_m_hh"]
df["After raise"] = kindergeld_new["kindergeld__betrag_m_hh"]
df["hh_id"] = data["hh_id"]
df = df.drop_duplicates("hh_id").set_index("hh_id")
df["Income (per household)"] = total_income_m_hh

We can illustrate the effect of the special child bonus of 2020 by plotting the monthly *Kindergeld* against the income on household level.

In [ ]:
fig = px.line(
    data_frame=df,
    x="Income (per household)",
    y=["Status Quo", "After raise"],
    title="Effects of hypothetical raise of Kindergeld",
)
fig.update_layout(
    xaxis_title="Monthly gross income in € (per household)",
    yaxis_title="Kindergeld in € per month",
)

Unsurprisingly, the policy reform increases the amount of money that households receive if they are entitled to *Kindergeld*.

Of course this is a *very* basic visualization and analysis. Have a look at the [policy functions tutorial](policy_functions.ipynb) to see a more sophisticated approach for this example taking into account the interaction with ALG-II. At the same time you will learn how to change and add functions to a policy environment!
